In [148]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [149]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.6.0


In [150]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [151]:
from azureml.core import Workspace, Dataset

from azureml.core.authentication import ServicePrincipalAuthentication
 
svc_pr_password = "1fY58u0dpP1Yg-i.A~rUp_iz04RxWUFSwv"
 
svc_pr = ServicePrincipalAuthentication(
    tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47",
    service_principal_id="8a3ddafe-6dd6-48af-867e-d745232a1833",
    service_principal_password="1fY58u0dpP1Yg-i.A~rUp_iz04RxWUFSwv")
 
ws = Workspace(
    subscription_id="c46a9435-c957-4e6c-a0f4-b9a597984773",
    resource_group="mlops",
    workspace_name="gputraining",
    auth=svc_pr
    )
ws

Workspace.create(name='gputraining', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops')

In [129]:
import urllib.request

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
                           filename=os.path.join(data_folder, 'train-images-idx3-ubyte.gz'))
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
                           filename=os.path.join(data_folder, 'train-labels-idx1-ubyte.gz'))
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
                           filename=os.path.join(data_folder, 't10k-images-idx3-ubyte.gz'))
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz',
                           filename=os.path.join(data_folder, 't10k-labels-idx1-ubyte.gz'))

('/mnt/batch/tasks/shared/LS_root/mounts/clusters/devcluster1/code/users/babal/parallelstep/tensorflow/data/t10k-labels-idx1-ubyte.gz',
 <http.client.HTTPMessage at 0x7efc2414c278>)

In [120]:

from azureml.core.dataset import Dataset
web_paths = ['https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [152]:
dataset_registered = False
try:
    temp = Dataset.get_by_name(workspace = ws, name = 'bbminsttest')
    dataset_registered = True
except:
    print("The dataset mnist-dataset is not registered in workspace yet.")

if not dataset_registered:
    dataset = dataset.register(workspace = ws,
                               name = 'bbminsttest',
                               description='training and test dataset',
                               create_new_version=True)
# list the files referenced by dataset
dataset.to_path()
dataset = temp

In [153]:
dataset.to_path()

['/img_1.jpg',
 '/img_10.jpg',
 '/img_100.jpg',
 '/img_101.jpg',
 '/img_102.jpg',
 '/img_103.jpg',
 '/img_104.jpg',
 '/img_105.jpg',
 '/img_106.jpg',
 '/img_107.jpg',
 '/img_108.jpg',
 '/img_109.jpg',
 '/img_11.jpg',
 '/img_110.jpg',
 '/img_111.jpg',
 '/img_112.jpg',
 '/img_113.jpg',
 '/img_114.jpg',
 '/img_115.jpg',
 '/img_116.jpg',
 '/img_117.jpg',
 '/img_118.jpg',
 '/img_119.jpg',
 '/img_12.jpg',
 '/img_120.jpg',
 '/img_121.jpg',
 '/img_122.jpg',
 '/img_123.jpg',
 '/img_124.jpg',
 '/img_125.jpg',
 '/img_126.jpg',
 '/img_127.jpg',
 '/img_128.jpg',
 '/img_129.jpg',
 '/img_13.jpg',
 '/img_130.jpg',
 '/img_131.jpg',
 '/img_132.jpg',
 '/img_133.jpg',
 '/img_134.jpg',
 '/img_135.jpg',
 '/img_136.jpg',
 '/img_137.jpg',
 '/img_138.jpg',
 '/img_139.jpg',
 '/img_14.jpg',
 '/img_140.jpg',
 '/img_141.jpg',
 '/img_142.jpg',
 '/img_143.jpg',
 '/img_144.jpg',
 '/img_145.jpg',
 '/img_146.jpg',
 '/img_147.jpg',
 '/img_148.jpg',
 '/img_149.jpg',
 '/img_15.jpg',
 '/img_150.jpg',
 '/img_151.jpg',
 '/im

In [154]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-07-16T17:37:55.640000+00:00', 'errors': None, 'creationTime': '2020-05-28T21:27:16.173526+00:00', 'modifiedTime': '2020-05-28T21:27:32.037057+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 8, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [155]:
def_data_store = ws.get_default_datastore()

In [156]:
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter

pipeline_param = PipelineParameter(name="mnist_param", default_value=dataset)
input_mnist_ds_consumption = DatasetConsumptionConfig("minist_param_config", pipeline_param).as_mount()

In [157]:
from azureml.pipeline.core import Pipeline, PipelineData

output_dir = PipelineData(name="inferences", 
                          datastore=def_data_store, 
                          output_path_on_compute="mnist/results")

In [158]:
from azureml.core import Model
from azureml.core.model import Model
from azureml.core import Run
from azureml.core.dataset import Dataset

from azureml.core import Workspace, Dataset

from azureml.core.authentication import ServicePrincipalAuthentication

In [159]:
model_path = Model.get_model_path("tf-dnn-mnist",6,ws)

In [160]:
model_path

'azureml-models/tf-dnn-mnist/6/model'

In [161]:
import tensorflow as tf
imported_model = tf.saved_model.load(model_path)

In [162]:
#import tensorflow as tf
#imported_model = tf.saved_model.load('./model')

In [163]:
imported_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7efc1c58b748>

In [164]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

batch_conda_deps = CondaDependencies.create(pip_packages=["tensorflow==2.0.0", "pillow",
                                                          "azureml-core", "azureml-dataprep[pandas, fuse]"])

batch_env = Environment(name="batch_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_GPU_IMAGE

In [165]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    source_directory='.',
    entry_script="batchscore3.py",
    mini_batch_size=PipelineParameter(name="batch_size_param", default_value="5"),
    error_threshold=10,
    output_action="append_row",
    append_row_file_name="mnist_outputs.txt",
    environment=batch_env,
    compute_target=compute_target,
    process_count_per_node=PipelineParameter(name="process_count_param", default_value=2),
    node_count=2)

In [166]:
from azureml.pipeline.steps import ParallelRunStep
from datetime import datetime

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

In [167]:
from azureml.pipeline.steps import ParallelRunStep

parallelrun_step = ParallelRunStep(
    name=parallel_step_name,
    parallel_run_config=parallel_run_config,
    inputs=[input_mnist_ds_consumption],
    output=output_dir,
    allow_reuse=True
)

In [168]:
from azureml.pipeline.core import Pipeline
from azureml.core.experiment import Experiment

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
experiment = Experiment(ws, 'batch_scoring')
pipeline_run = experiment.submit(pipeline)

Created step batchscoring-202007161814 [21d40cf5][3c9df4d6-3dd8-4888-a9cb-27c7ae45989a], (This step will run and generate new outputs)
Submitted PipelineRun 9462637d-545c-456d-b303-e535d5429f2f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/9462637d-545c-456d-b303-e535d5429f2f?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/gputraining


In [169]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

pipeline_run.wait_for_completion(show_output=True)

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', '…

PipelineRunId: 9462637d-545c-456d-b303-e535d5429f2f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/9462637d-545c-456d-b303-e535d5429f2f?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/gputraining
PipelineRun Status: Running


StepRunId: bbb6b3c5-7e89-4375-9dfa-cab5cdaf1d1f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/bbb6b3c5-7e89-4375-9dfa-cab5cdaf1d1f?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/gputraining
StepRun( batchscoring-202007161814 ) Status: NotStarted
StepRun( batchscoring-202007161814 ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_5be0a9a4403b421129b3f6529de5977db46c90855968f3c920e5d435f6a1b23c_d.txt
2020-07-16T18:18:08Z Executing 'Copy ACR Details file' on 10.0.0.5
2020-07-16T18:18:08Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-07-16T18:18:08Z Starting output-watcher...

'Finished'